# DATA PREPROCESSING AND DATA WRANGLING

In [1]:
import numpy as np
import pandas as pd

#import json

In [2]:
dataset= pd.read_pickle('data/pickles/df_new')
dataset.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,matchCVV
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,06/2023,2015-03-14,2015-03-14,1803,PURCHASE,0.0,False,False,False,1
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,02/2024,2015-03-14,2015-03-14,767,PURCHASE,0.0,True,False,False,1
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,08/2025,2015-03-14,2015-03-14,767,PURCHASE,0.0,False,False,False,1
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,08/2025,2015-03-14,2015-03-14,767,PURCHASE,0.0,False,False,False,1
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,10/2029,2015-08-06,2015-08-06,3143,PURCHASE,0.0,True,False,False,1


### Account Number , Customer Id and Card Last 4 digits

In [3]:
#The Two columns are similar and moreover as they are simply identifications, they will not influence Predictions
#We can drop them 

df_prep= dataset.copy()
df_prep.drop(['accountNumber','customerId','cardLast4Digits'], axis=1, inplace=True)


In [4]:
df_prep[df_prep.isFraud == 0]

,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,matchCVV
0,5000,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,PURCHASE,0.00,False,False,False,1
1,5000,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,PURCHASE,0.00,True,False,False,1
2,5000,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,PURCHASE,0.00,False,False,False,1
3,5000,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,PURCHASE,0.00,False,False,False,1
4,5000,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,10/2029,2015-08-06,2015-08-06,PURCHASE,0.00,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,50000,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,90,01,rideshare,12/2022,2012-08-23,2012-08-23,PURCHASE,1095.04,False,False,False,1
786359,50000,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,09,01,online_subscriptions,08/2023,2012-08-23,2012-08-23,PURCHASE,1214.96,False,False,False,1
786360,50000,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,02,01,rideshare,08/2025,2012-08-23,2012-08-23,PURCHASE,1233.85,False,False,False,1
786361,50000,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,09,99,online_retail,07/2022,2012-08-23,2012-08-23,PURCHASE,1283.28,False,False,False,1


### Acquired Country and Merchant Country Code

In [5]:
# check bot of them how many are unique

count_acqCountry = df_prep['acqCountry'].unique()
count_acqCountry=len(count_acqCountry)
count_merchantCountryCode =len(df_prep['merchantCountryCode'].unique())

print(count_acqCountry)
print(count_merchantCountryCode)

4
4


In [6]:
#The Two Fields both have similar values with only 4 uniques, I presume they will not affect our model
#Hence I will remove these fields entirely 

df_prep.drop(['acqCountry','merchantCountryCode'], axis=1, inplace=True)

### Dealing with Date Time

The Transaction Date Time Column is very important to this Dataset. I will convert the Datatime into an epoch.

This will allow us to split the Date Time into chunks

There are also some relative Date Time which map the days since an event. I would like to represent them in Epoch Time as well.

In [7]:
#I reformat the Transaction Date Time, I format the date into the form of epoch time 
#We also create new fields that will store each of our formatted dates : Year, Month, Day etc 

def DateTimeChange(df):
    df['transDT']= pd.to_datetime(df['transactionDateTime'])
    df['transMonth']= df['transDT'].dt.month.astype(int)
    df['transDay'] = df['transDT'].dt.day.astype(int)
    df['transYear']= df['transDT'].dt.year.astype(int)
    df['transHour']= df['transDT'].dt.hour.astype(int)
    df['transactionformatted'] = df['transDT'].apply(lambda x: int(x.timestamp()))
    
    df.drop(['transactionDateTime','transDT'], axis=1, inplace=True)
    return df


In [8]:
#Here I repeat the above procedure for the relative dates in the dataframe that captures 'days since' type of dates.

def rel_time_change(df):
    df["dateOfLastAddressChange"] = pd.to_datetime(df["dateOfLastAddressChange"])
    df["accountOpenDate"] = pd.to_datetime(df["accountOpenDate"])
    df["currentExpDate"] = pd.to_datetime(df["currentExpDate"])
    
    df["dateOfLastAddressChangeformatted"] = df["dateOfLastAddressChange"].apply(lambda x: int(x.timestamp()))
    df["accountOpenDateformatted"] = df["accountOpenDate"].apply(lambda x: int(x.timestamp()))
    df["currentExpDateformatted"] = df["currentExpDate"].apply(lambda x: int(x.timestamp()))
    
    df["timeTillExp"] = df["currentExpDateformatted"] - df["transactionformatted"]
    df["timeSinceAccountOpening"] = df["transactionformatted"] - df["accountOpenDateformatted"]
    df["timeSinceAddressChange"] = df["transactionformatted"] - df["dateOfLastAddressChangeformatted"]
    
    df.drop(['dateOfLastAddressChange','accountOpenDate','currentExpDate','dateOfLastAddressChangeformatted','accountOpenDateformatted','currentExpDateformatted'], axis=1, inplace=True)
    
    return df

In [9]:
df_prep= DateTimeChange(df_prep)


In [10]:
df_prep = rel_time_change(df_prep)

C:\Users\dbavi\AppData\Local\Temp\ipykernel_11516\2972419366.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["currentExpDate"] = pd.to_datetime(df["currentExpDate"])


In [11]:
df_prep.columns

Index(['creditLimit', 'availableMoney', 'transactionAmount', 'merchantName',
       'posEntryMode', 'posConditionCode', 'merchantCategoryCode',
       'transactionType', 'currentBalance', 'cardPresent',
       'expirationDateKeyInMatch', 'isFraud', 'matchCVV', 'transMonth',
       'transDay', 'transYear', 'transHour', 'transactionformatted',
       'timeTillExp', 'timeSinceAccountOpening', 'timeSinceAddressChange'],
      dtype='object')

In [12]:
# df_prep = rel_time_change(df_prep)
df_prep.head()

,creditLimit,availableMoney,transactionAmount,merchantName,posEntryMode,posConditionCode,merchantCategoryCode,transactionType,currentBalance,cardPresent,...,isFraud,matchCVV,transMonth,transDay,transYear,transHour,transactionformatted,timeTillExp,timeSinceAccountOpening,timeSinceAddressChange
0,5000,5000.0,98.55,Uber,02,01,rideshare,PURCHASE,0.0,False,...,False,1,8,13,2016,14,1471098452,214479148,44807252,44807252
1,5000,5000.0,74.51,AMC #191138,09,01,entertainment,PURCHASE,0.0,True,...,False,1,10,11,2016,5,1476162354,230583246,49871154,49871154
2,5000,5000.0,7.47,Play Store,09,01,mobileapps,PURCHASE,0.0,False,...,False,1,11,8,2016,9,1478596719,275409681,52305519,52305519
3,5000,5000.0,7.47,Play Store,09,01,mobileapps,PURCHASE,0.0,False,...,False,1,12,10,2016,2,1481336090,272670310,55044890,55044890
4,5000,5000.0,71.18,Tim Hortons #947751,02,01,fastfood,PURCHASE,0.0,True,...,False,1,3,24,2016,21,1458853486,426653714,20034286,20034286


### Dealing with Booleans

Convert all boolean features to binary 1s and 0s

cardPresent, expirationDateKeyInMatch, isFraud are the Boolean features we will convert to Binary

In [13]:
def bool_to_binary(df):
    features=['isFraud','cardPresent','expirationDateKeyInMatch']
    df[features]= df[features].astype(int)
    return df

df_prep = bool_to_binary(df_prep)

In [14]:
df_prep

,creditLimit,availableMoney,transactionAmount,merchantName,posEntryMode,posConditionCode,merchantCategoryCode,transactionType,currentBalance,cardPresent,...,isFraud,matchCVV,transMonth,transDay,transYear,transHour,transactionformatted,timeTillExp,timeSinceAccountOpening,timeSinceAddressChange
0,5000,5000.00,98.55,Uber,02,01,rideshare,PURCHASE,0.00,0,...,0,1,8,13,2016,14,1471098452,214479148,44807252,44807252
1,5000,5000.00,74.51,AMC #191138,09,01,entertainment,PURCHASE,0.00,1,...,0,1,10,11,2016,5,1476162354,230583246,49871154,49871154
2,5000,5000.00,7.47,Play Store,09,01,mobileapps,PURCHASE,0.00,0,...,0,1,11,8,2016,9,1478596719,275409681,52305519,52305519
3,5000,5000.00,7.47,Play Store,09,01,mobileapps,PURCHASE,0.00,0,...,0,1,12,10,2016,2,1481336090,272670310,55044890,55044890
4,5000,5000.00,71.18,Tim Hortons #947751,02,01,fastfood,PURCHASE,0.00,1,...,0,1,3,24,2016,21,1458853486,426653714,20034286,20034286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,50000,48904.96,119.92,Lyft,90,01,rideshare,PURCHASE,1095.04,0,...,0,1,12,22,2016,18,1482432252,187420548,136752252,136752252
786359,50000,48785.04,18.89,hulu.com,09,01,online_subscriptions,PURCHASE,1214.96,0,...,0,1,12,25,2016,16,1482682834,208165166,137002834,137002834
786360,50000,48766.15,49.43,Lyft,02,01,rideshare,PURCHASE,1233.85,0,...,0,1,12,27,2016,15,1482853584,271152816,137173584,137173584
786361,50000,48716.72,49.89,walmart.com,09,99,online_retail,PURCHASE,1283.28,0,...,0,1,12,29,2016,0,1482971455,173662145,137291455,137291455


### One Hot-Encode Categorical Variables

In [17]:
def encode_and_bind(df, feature_to_encode):
    for feature in feature_to_encode:
        dummies = pd.get_dummies(df[feature])
        dummies.rename(columns=lambda x: feature + '_' + str(x), inplace=True)
        df = pd.concat([df, dummies], axis=1)        
        
    return(df)

In [20]:
#Process the Merchant Name Column, Keep the name by deleting everything after the #, also remove trailing whitespace
df_prep["merchantName"] = df_prep["merchantName"].str.split('#').str[0].str.strip()

# define the list of categorical features to encode
feature_to_encode = ['posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'transactionType', 'merchantName']

# one-hot encode the categorical features and bind the encoded features to the original DataFrame
df_prep = encode_and_bind(df_prep, feature_to_encode)

df_prep

,creditLimit,availableMoney,transactionAmount,merchantName,posEntryMode,posConditionCode,merchantCategoryCode,transactionType,currentBalance,cardPresent,...,merchantName_ikea.com,merchantName_netflix.com,merchantName_oldnavy.com,merchantName_pottery-barn.com,merchantName_sears.com,merchantName_staples.com,merchantName_target.com,merchantName_walmart.com,merchantName_westelm.com,merchantName_williamssonoma.com
0,5000,5000.00,98.55,Uber,02,01,rideshare,PURCHASE,0.00,0,...,False,False,False,False,False,False,False,False,False,False
1,5000,5000.00,74.51,AMC,09,01,entertainment,PURCHASE,0.00,1,...,False,False,False,False,False,False,False,False,False,False
2,5000,5000.00,7.47,Play Store,09,01,mobileapps,PURCHASE,0.00,0,...,False,False,False,False,False,False,False,False,False,False
3,5000,5000.00,7.47,Play Store,09,01,mobileapps,PURCHASE,0.00,0,...,False,False,False,False,False,False,False,False,False,False
4,5000,5000.00,71.18,Tim Hortons,02,01,fastfood,PURCHASE,0.00,1,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,50000,48904.96,119.92,Lyft,90,01,rideshare,PURCHASE,1095.04,0,...,False,False,False,False,False,False,False,False,False,False
786359,50000,48785.04,18.89,hulu.com,09,01,online_subscriptions,PURCHASE,1214.96,0,...,False,False,False,False,False,False,False,False,False,False
786360,50000,48766.15,49.43,Lyft,02,01,rideshare,PURCHASE,1233.85,0,...,False,False,False,False,False,False,False,False,False,False
786361,50000,48716.72,49.89,walmart.com,09,99,online_retail,PURCHASE,1283.28,0,...,False,False,False,False,False,False,False,True,False,False


In [21]:
df_prep.drop(feature_to_encode, axis=1, inplace=True)

In [22]:
pd.to_pickle(df_prep,'data/pickles/df_prep', protocol=4)

### WRANGLING

In [45]:
df_dup=dataset.copy()

In [46]:
df_dup = DateTimeChange(df_dup)

In [47]:
df_dup= df_dup.sort_values(['accountNumber'])
df_dup.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,...,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,matchCVV,transMonth,transDay,transYear,transHour,transactionformatted
541974,100088067,100088067,50000,49424.61,10.18,Best Pub,US,US,02,01,...,575.39,True,False,False,1,12,28,2016,12,1482928029
541919,100088067,100088067,50000,50000.00,3.96,Apple iTunes,US,US,09,01,...,0.00,False,False,False,1,5,5,2016,22,1462488470
541920,100088067,100088067,50000,49996.04,5.26,1st Sandwitch Bar #758805,US,US,02,01,...,3.96,True,False,False,1,5,6,2016,2,1462503241
541921,100088067,100088067,50000,49990.78,232.47,Blue Mountain eCards,US,US,09,01,...,9.22,False,False,False,1,5,11,2016,12,1462971326
541922,100088067,100088067,50000,49758.31,71.41,Next Day eCards,US,US,05,01,...,241.69,False,False,False,1,5,16,2016,20,1463430780


In [48]:
df_dup = df_dup.reset_index(drop=True)
df_dup.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,...,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,matchCVV,transMonth,transDay,transYear,transHour,transactionformatted
0,100088067,100088067,50000,49424.61,10.18,Best Pub,US,US,02,01,...,575.39,True,False,False,1,12,28,2016,12,1482928029
1,100088067,100088067,50000,50000.00,3.96,Apple iTunes,US,US,09,01,...,0.00,False,False,False,1,5,5,2016,22,1462488470
2,100088067,100088067,50000,49996.04,5.26,1st Sandwitch Bar #758805,US,US,02,01,...,3.96,True,False,False,1,5,6,2016,2,1462503241
3,100088067,100088067,50000,49990.78,232.47,Blue Mountain eCards,US,US,09,01,...,9.22,False,False,False,1,5,11,2016,12,1462971326
4,100088067,100088067,50000,49758.31,71.41,Next Day eCards,US,US,05,01,...,241.69,False,False,False,1,5,16,2016,20,1463430780


In [49]:
time_limit =300  # 300 seconds ie 5 minutes

In [50]:
df_grouped = df_dup.groupby('accountNumber')
df_grouped

In [51]:
reversed_trans = []
multiswipe_trans = []

In [52]:
for accNo, df in df_grouped:
    df = df.sort_values(['transactionAmount'])
    df = df.reset_index(drop=True)
    
    # Loop through each transaction, comparing it to every other transaction with a higher index
    for i in range(0,len(df["transactionAmount"])-1):
        for j in range(i+1,len(df["transactionAmount"])-1):
            
            # If two transactions have the same amount, calculate the time difference between them
            if df_dup ["transactionAmount"][i] == df_dup ["transactionAmount"][j]:
                
                time_difference = abs(df_dup ["transactionformatted"][j] - df_dup ["transactionformatted"][i])
            
                # If one transaction is a REVERSAL and the other is not,
                #consider the REVERSAL transaction as a potential Duplicate
                if (df_dup ["transactionType"][j] == "REVERSAL" and \
                    df_dup ["transactionType"][i] != "REVERSAL") \
                or (df_dup ["transactionType"][j] != "REVERSAL" \
                    and df_dup ["transactionType"][i] == "REVERSAL"):
                    
                    reversed_trans.append(df_dup ["transactionAmount"][j])

            
                # If the time difference between two transactions is less than or equal to a certain limit, it is a duplicate               
                elif time_difference <= time_limit:
                    
                    multiswipe_trans.append(df_dup ["transactionAmount"][j])
                    
            # The Transactions are sorted by Amount, if the amount is lesser, break out of the loop            
            elif df_dup ["transactionAmount"][i] < df_dup ["transactionAmount"][j]:
                
                break

In [53]:
reversed_count = len(reversed_trans)
reversed_total = round(np.array(reversed_trans).sum(),4)
multiswipe_count = len(multiswipe_trans)
multiswipe_total = round(np.array(multiswipe_trans).sum(),4)

print("There are total of {} reversed transactions amounting to ${}".format(reversed_count, reversed_total))
print("There are total of {} multiswipe transactions amounting to ${}".format(multiswipe_count, multiswipe_total))

There are total of 3501 reversed transactions amounting to $640960.33
There are total of 7649 multiswipe transactions amounting to $2690437.35


In [55]:
df_dup['timeSinceAccountOpening']

KeyError: 'timeSinceAccountOpening'